In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

import xarray as xr

from shapely.wkt import loads
from shapely.geometry import Polygon
from shapely.geometry import Point

import rasterio
import rasterio.sample

from functools import partial
from scipy.spatial.distance import cdist

In [2]:
parquet_files = {'marine_protected_areas': ('../data/PR100/Habitat/marine_protected_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'land_nationwide_hapc': ('../data/PR100/Habitat/land_nationwide_hapc.parquet',
  'EPSG:32161',
  'centroid'),
 'artificial_reefs': ('../data/PR100/Habitat/artificial_reefs.parquet',
  'EPSG:32161',
  'irrelevant'),
 'nationwide_hapc': ('../data/PR100/Habitat/nationwide_hapc.parquet',
  'EPSG:32161',
  'irrelevant'),
 'areas_protegidas_agencia': ('../data/PR100/Habitat/areas_protegidas_agencia.parquet',
  'EPSG:32161',
  'centroid'),
 'noaa_charted_submarine_cables_30m_buffer': ('../data/PR100/Infrastructure/noaa_charted_submarine_cables_30m_buffer.parquet',
  'EPSG:32161',
  'irrelevant'),
 'setbacks_xmission': ('../data/PR100/Infrastructure/setbacks_xmission.parquet',
  'EPSG:32161',
  'lines'),
 'submarine_cable_30m_buffer': ('../data/PR100/Infrastructure/submarine_cable_30m_buffer.parquet',
  'EPSG:32161',
  'irrelevant'),
 'pipeline_areas': ('../data/PR100/Infrastructure/pipeline_areas.parquet',
  'EPSG:32161',
  'irrelevant'),
 'setbacks_roads': ('../data/PR100/Infrastructure/setbacks_roads.parquet',
  'EPSG:4326',
  'density'),
 'setbacks_runway': ('../data/PR100/Infrastructure/setbacks_runway.parquet',
  'EPSG:4326',
  'centroid'),
 'setbacks_airport': ('../data/PR100/Infrastructure/setbacks_airport.parquet',
  'EPSG:4326',
  'points'),
 'setbacks_buildings': ('../data/PR100/Infrastructure/setbacks_buildings.parquet',
  'EPSG:4326',
  'density'),
 'federal_and_state_waters': ('../data/PR100/Boundaries/federal_and_state_waters.parquet',
  'EPSG:32161',
  'centroid'),
 'coastal_zone_mgmt_act_boundary': ('../data/PR100/Boundaries/coastal_zone_mgmt_act_boundary.parquet',
  'EPSG:32161',
  'centroid'),
 'land_protected_areas': ('../data/PR100/Boundaries/land_protected_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'water_bodies': ('../data/PR100/Topography/water_bodies.parquet',
  'EPSG:4326',
  'centroid'),
 'wrecks_and_obstructions': ('../data/PR100/Hazards/wrecks_and_obstructions.parquet',
  'EPSG:4326',
  'points'),
 'ocean_disposal_sites': ('../data/PR100/Hazards/ocean_disposal_sites.parquet',
  'EPSG:32161',
  'centroid'),
 'tropical_cyclone_storm_segments': ('../data/PR100/Hazards/tropical_cyclone_storm_segments.parquet',
  'EPSG:32161',
  'lines'),
 'tropical_cyclone_wind_exposure': ('../data/PR100/Hazards/tropical_cyclone_wind_exposure.parquet',
  'EPSG:32161',
  'wind_exposure'),
 'unexploded_ordinance_areas_fuds': ('../data/PR100/Hazards/unexploded_ordinance_areas_fuds.parquet',
  'EPSG:32161',
  'irrelevant'),
 'danger_zones_and_restricted_areas': ('../data/PR100/Hazards/danger_zones_and_restricted_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'unexploded_ordinance_areas': ('../data/PR100/Hazards/unexploded_ordinance_areas.parquet',
  'EPSG:4326',
  'centroid')}

In [3]:
tif_files = ['../data/wind_atlas/PRI_wind-speed_100m.tif',
 '../data/solar_atlas/GHI.tif',
 '../data/PR100/Topography/slope.tif',
 '../data/PR100/Topography/elevation.tif']

In [4]:
binary_mask_columns = ['marine_protected_areas', 'land_nationwide_hapc',
       'areas_protegidas_agencia', 'setbacks_xmission',
       'setbacks_runway', 'setbacks_airport',
       'federal_and_state_waters', 'coastal_zone_mgmt_act_boundary',
       'land_protected_areas', 'water_bodies', 'wrecks_and_obstructions',
       'ocean_disposal_sites', 'danger_zones_and_restricted_areas',
       'unexploded_ordinance_areas']

In [5]:
def find_parquet_files(directory):
    """
    Find all Parquet files within a directory and its subdirectories.

    :param directory: The base directory to search.
    :return: A list of paths to Parquet files.

    Note: This function isn't used in this notebook, because the file list
    has already been generated and enriched (see 'parquet_files' dict above)
    """
    parquet_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.parquet'):
                parquet_files.append(os.path.join(root, file))
    return parquet_files

In [6]:
def find_tif_files(directory):
    """
    Find all TIF files within a directory and its subdirectories.

    :param directory: The base directory to search.
    :return: A list of paths to TIF files.

    Note: This function isn't used in this notebook, because the file list
    has already been generated (see 'tif_files' dict above)
    """
    tif_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tif'):
                tif_files.append(os.path.join(root, file))
    return tif_files

In [7]:
def get_gdf(parquet_file, crs, grid):
    df = pd.read_parquet(parquet_file)
    df['geometry'] = df['geometry'].apply(loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.set_crs(crs, inplace=True)
    gdf.to_crs(grid.crs, inplace=True)
    return gdf

In [8]:
def create_grid(x_start, x_end, y_start, y_end, square_size):
    grid = []
    for x in np.arange(x_start, x_end, square_size):
        for y in np.arange(y_start, y_end, square_size):
            # Define the square's coordinates
            square = Polygon([(x, y), (x + square_size, y), (x + square_size, y + square_size), (x, y + square_size)])
            grid.append(square)
    return gpd.GeoDataFrame({'geometry': grid}, crs="EPSG:32161")

In [9]:
def calculate_cyclone_failure_percentage(df):
    probability_of_hurricane_occuring = df['occurrence64ktMajorHurricanes'] / 117
    probability_of_failure_due_to_hurricane = 0.5
    average_life_span = 25
    df['turbine_cyclone_failure_percentage'] = probability_of_hurricane_occuring * probability_of_failure_due_to_hurricane * average_life_span / 2
    return df

In [10]:
def rasterize_cyclone_failure_risk(grid_gdf, cyclone_data_gdf):
    # Ensure both dataframes are in the same CRS
    cyclone_data_gdf = cyclone_data_gdf.to_crs(grid_gdf.crs)

    # Spatial join - to associate cyclone data with each grid cell
    joined_gdf = gpd.sjoin(grid_gdf, cyclone_data_gdf, how='left', predicate='intersects')

    # Apply the failure cost calculation to each row
    joined_gdf = calculate_cyclone_failure_percentage(joined_gdf)

    # Group by grid cell index and average the scores within each cell
    aggregated_scores = joined_gdf.groupby(joined_gdf.index)['turbine_cyclone_failure_percentage'].mean()

    return aggregated_scores.reindex(grid_gdf.index, fill_value=0)

In [11]:
def rasterize(grid, data, type):
    joined_gdf = gpd.sjoin(grid, data[['geometry']], how='right', predicate='intersects')
    count = joined_gdf.groupby('index_left').size().reindex(grid.index, fill_value=0)
    if type == 'density':
        return count
    else:
        return count > 0

In [12]:
def process_parquet_file(file_path, crs, type, handle, grid):
    gdf = get_gdf(file_path, crs, grid)
    if type == 'irrelevant':
        pass
    elif type == 'wind_exposure':
        grid[handle] = rasterize_cyclone_failure_risk(grid, gdf)
    else:
        grid[handle] = rasterize(grid, gdf, type)

In [13]:
def process_parquet_files(files, grid):
    for handle, file_data in files.items():
        file_path = file_data[0]
        crs = file_data[1]
        type = file_data[2]
        print(f'Processing file: {file_path}')
        process_parquet_file(file_path, crs, type, handle, grid)

In [14]:
def process_tif_file(file_path, grid, coords):
    handle = file_path.split('/')[-1].split('.')[0]
    # Open the raster and sample values at each coordinate
    with rasterio.open(file_path) as src:
        if grid.crs != src.crs:
            centroids = grid.geometry.centroid.to_crs(src.crs)
            coords = [(x,y) for x, y in zip(centroids.geometry.x, centroids.geometry.y)]

        # Sample the raster at each centroid location
        values = [x[0] for x in rasterio.sample.sample_gen(src, coords)]
    
    # Assign the values to the grid dataframe
    grid[handle] = values

In [15]:
def process_hourly_file(grid, hourly):
    # Ensure both DataFrames are in the same CRS
    if grid.crs != hourly.crs:
        hourly_wind = hourly.to_crs(grid.crs)

    # Iterate over the columns in hourly and assign values to grid
    for column in hourly.columns:
        joined = gpd.sjoin_nearest(grid, hourly[[column, 'geometry']], how='left')
        grid[column] = joined[column]

    return grid

In [16]:
def process_tif_files(files, grid):
    # Compute centroids
    centroids = grid.geometry.centroid
    # Create an array of coordinates in (x, y) format
    coords = [(x,y) for x, y in zip(centroids.geometry.x, centroids.geometry.y)]

    for file_path in tif_files:
        print(f'Processing file: {file_path}')
        process_tif_file(file_path, grid, coords)

In [17]:
def plot_columns(gdf, columns=[]):
    if len(columns) == 0:
        columns = gdf.columns
    for column in columns:
        if column in ['geometry', 'centroid']:
            continue
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        gdf.plot(column=column, ax=ax, legend=True)
        plt.title(column)
        plt.show()

In [18]:
environment_gdf = create_grid(100000, 300000, 200000, 300000, 500)

In [19]:
process_parquet_files(parquet_files, environment_gdf)

Processing file: ../data/PR100/Habitat/marine_protected_areas.parquet
Processing file: ../data/PR100/Habitat/land_nationwide_hapc.parquet
Processing file: ../data/PR100/Habitat/artificial_reefs.parquet
Processing file: ../data/PR100/Habitat/nationwide_hapc.parquet
Processing file: ../data/PR100/Habitat/areas_protegidas_agencia.parquet
Processing file: ../data/PR100/Infrastructure/noaa_charted_submarine_cables_30m_buffer.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_xmission.parquet
Processing file: ../data/PR100/Infrastructure/submarine_cable_30m_buffer.parquet
Processing file: ../data/PR100/Infrastructure/pipeline_areas.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_roads.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_runway.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_airport.parquet
Processing file: ../data/PR100/Infrastructure/setbacks_buildings.parquet
Processing file: ../data/PR100/Boundaries/federal_and_state_wat

In [20]:
process_tif_files(tif_files, environment_gdf)

Processing file: ../data/wind_atlas/PRI_wind-speed_100m.tif
Processing file: ../data/solar_atlas/GHI.tif
Processing file: ../data/PR100/Topography/slope.tif
Processing file: ../data/PR100/Topography/elevation.tif


In [21]:
environment_gdf['masked'] = environment_gdf[binary_mask_columns].any(axis=1)

In [22]:
environment_gdf['water'] = environment_gdf[['federal_and_state_waters', 'water_bodies']].any(axis=1)

In [23]:
environment_gdf['centroid'] = environment_gdf.centroid

In [24]:
environment_gdf['occupied'] = 0

## Calculating per-cell energy demand

In [25]:
# Load the demand profile data
demand_df = pd.read_csv('../data/generation_and_demand/demand_profile.csv')

# Convert the DataFrame to a dictionary with 'hour' as keys and 'demand_MW' as values
demand_dict = dict(zip(demand_df['local_hour'], demand_df['demand_MW']))

# # Calculate total building density
# total_building_density = environment_gdf['setbacks_buildings'].sum()

# # Loop through each hour in the demand dictionary and create a corresponding demand column
# for hour, hour_demand in demand_dict.items():
#     # Create a new column for this hour's demand
#     environment_gdf[f'demand_{hour}'] = hour_demand * environment_gdf['setbacks_buildings'] / total_building_density



## Calculating dist to nearest transmission line

In [26]:
def nearest_transmission_distance(row, gdf):
    if row['setbacks_xmission']:
        return 0
    else:
        # Calculate distances to all cells where setbacks_xmission is True
        distances = gdf[gdf['setbacks_xmission']]['geometry'].distance(row['geometry'])
        # Return the minimum distance
        return distances.min()

In [27]:
environment_gdf['distance_to_transmission_line'] = environment_gdf.apply(nearest_transmission_distance, gdf=environment_gdf, axis=1)

# Incorporating ERA5 Wind Speed Data

This is data I saved off from an ERA5/Copernicus dataset. I grouped 365 days by lat-lon-hour_of_day to get diurnal wind curves. 
These are available at 25km resolution, so pretty low res. For that reason, we don't use the actual 100m wind speed values. 
Instead, we calculate a noramalized hourly coefficien, which we multiply with 100m wind speed data from the Global Wind Atlas to 
get an estimate of diurnal wind speed variability using the higher resolution GWA data. 

In [28]:
# Total wind speed is derived in this dataset. This cell extracts it and shows what the data looks like. 
ds = xr.open_dataset("../data/generation_and_demand/2019_wind_speeds.nc")
total_wind_speed_array = ds.total_wind_speed
total_wind_speed_np = np.array(total_wind_speed_array)
print(total_wind_speed_np.shape)

(24, 7, 21)


In [29]:
def utc_to_pr_local_time(hour):
    return hour - 4 if hour > 4 else hour + 20

In [30]:
# Loop through lat, lon, and hr to get normalized values

hrs = np.array(ds.coords['hour'])
lats = np.array(ds.coords['latitude'])
lons = np.array(ds.coords['longitude'])

lists = []
for lat_idx in range(0, ds.coords['latitude'].shape[0]):
    for lon_idx in range(0, ds.coords['longitude'].shape[0]):
        hour_wind_speeds = {}
    
        this_point_wind_speeds = total_wind_speed_np[:, lat_idx, lon_idx]
        this_normalized_wind_speeds = this_point_wind_speeds/np.mean(this_point_wind_speeds)
        
        for hr_idx in range(0, ds.coords['hour'].shape[0]):
            this_point = Point(lons[lon_idx], lats[lat_idx])
    
            this_hour = hrs[hr_idx]

            if this_hour == 0:
                this_hour = 24

            this_hour = utc_to_pr_local_time(this_hour)

            hour_wind_speeds[f'normalized_wind_speed_hour_{this_hour}'] = this_normalized_wind_speeds[hr_idx]

        lists.append({'latitude': lats[lat_idx], 'longitude': lons[lon_idx], 'geometry': this_point, **hour_wind_speeds})

diurnal_wind_speed_gdf = gpd.GeoDataFrame(lists)
diurnal_wind_speed_gdf.set_crs(epsg=4326, inplace=True)
diurnal_wind_speed_gdf.to_crs(epsg=32161, inplace=True)

In [31]:
environment_gdf = gpd.sjoin_nearest(environment_gdf, diurnal_wind_speed_gdf.drop(columns=['latitude', 'longitude']), how='left').drop(columns=["index_right"])

normalized_wind_speed_columns = [col for col in environment_gdf.columns.values if 'normalized_wind_speed_hour_' in col]
raw_wind_speed_columns = [col.lstrip('normalized_') for col in normalized_wind_speed_columns]

environment_gdf[raw_wind_speed_columns] = environment_gdf[normalized_wind_speed_columns].mul(environment_gdf['PRI_wind-speed_100m'], axis=0)
environment_gdf = environment_gdf.drop(columns=normalized_wind_speed_columns)

In [32]:
# https://en.wind-turbine-models.com/turbines/1345-vestas-v126-3.0

power_curve = pd.read_csv('../data/generation_and_demand/V126_3MW_power_curve.csv')
power_curve = {row['wind_speed']: row['power_kW'] for idx, row in power_curve.iterrows()}

power_curve_key = np.array(list(power_curve.keys()))
power_curve_val = np.array(list(power_curve.values()))

power_curve_function = partial(np.interp, xp=power_curve_key, fp=power_curve_val, left=0, right=0)

power_columns = [col.replace('wind_speed_', 'wind_power_kW_') for col in raw_wind_speed_columns]

environment_gdf[power_columns] = environment_gdf[raw_wind_speed_columns].map(power_curve_function) * 4
environment_gdf = environment_gdf.drop(columns=raw_wind_speed_columns)
environment_gdf[power_columns].describe()

environment_gdf['daily_wind_power_kW'] = environment_gdf['PRI_wind-speed_100m'].map(power_curve_function) * 4

# Calculating Hourly Solar Data

In [33]:
hourly_solar_gdf = gpd.read_parquet('../data/generation_and_demand/pr_hourly_ghi_normalized.parquet')

In [34]:
hourly_solar_gdf.to_crs(environment_gdf.crs, inplace=True)

joined_gdf = gpd.sjoin_nearest(environment_gdf, hourly_solar_gdf, how='left')

# GHI is given in terms of daily totals of kWh/m^2
area = 1000**2
ground_coverage_factor = 0.5
solar_panel_efficiency = 0.2
inverter_efficiency = 0.95
hours_in_a_day = 24
GHI_to_kW = area * ground_coverage_factor * solar_panel_efficiency * inverter_efficiency / hours_in_a_day

# Multiplying the GHI with hourly factors
# Creating new columns for each hour
for hour in range(1, 25):
    hour = utc_to_pr_local_time(hour)
    hour_str = str(hour)
    environment_gdf[f'solar_power_kW_hour_{hour_str}'] = (joined_gdf['GHI'] * joined_gdf[hour_str] * GHI_to_kW).fillna(0)

environment_gdf['daily_solar_power_kW'] = environment_gdf['GHI'] * GHI_to_kW

In [35]:
environment_gdf.to_parquet('../data/processed/environment_gdf.parquet')

# Calculating Installation Costs

In [36]:
def calculate_installation_cost():
    # Wind operational expenses = $40/kW/yr
    # Wind turbine capacity is 3 MW
    # To get daily cost, multiply by 3000 (3 MW = 3000 kW) and divide by 365.25 days in a year
    wind_opex = 4 * 40 * 3000 / 365.25
    # Wind capital expenses = $1501/kW
    # To get daily cost, multiply by 3000 (3 MW = 3000 kW) and divide by (25 years of lifetime * 365.25 days in a year)
    wind_capex = 4 * 1501 * 3000 / (25 * 365.25)

    wind_daily_cost = wind_opex + wind_capex

    # Solar operational expenses = $16.12/kW/yr
    # Solar capacity is 20 MW, based on land footprint
    # To get daily cost, multiply by 20000 (20 MW = 20000 kW) and divide by 365.25 days in a year
    solar_opex = 4 * 16.12 * 20000 / 365.25
    # Solar capital expenses = $970/kW
    # Solar capacity is 20 MW, based on land footprint
    # To get daily cost, multiply by 20000 (20 MW = 20000 kW) and divide by (25 years of lifetime * 365.25 days in a year)
    solar_capex = 4 * 970 * 20000 / (25 * 365.25)

    solar_daily_cost = solar_opex + solar_capex

    return wind_daily_cost, solar_daily_cost

In [37]:
wind_daily_cost, solar_daily_cost = calculate_installation_cost()

In [38]:
wind_daily_cost, solar_daily_cost

(3286.735112936345, 12029.0212183436)

In [39]:
relevant_columns = ['geometry',
                    'masked', 
                    'occupied',
                    'slope',
                    'distance_to_transmission_line',
                    'tropical_cyclone_wind_exposure']
relevant_columns += [f'demand_{i}' for i in range(1,25)]
relevant_columns += [f'wind_power_kW_hour_{i}' for i in range(1,25)]
relevant_columns += [f'solar_power_kW_hour_{i}' for i in range(1,25)]

state = environment_gdf[relevant_columns].copy()
state.rename(columns={'tropical_cyclone_wind_exposure': 'cyclone_risk'}, inplace=True)

KeyError: "['demand_1', 'demand_2', 'demand_3', 'demand_4', 'demand_5', 'demand_6', 'demand_7', 'demand_8', 'demand_9', 'demand_10', 'demand_11', 'demand_12', 'demand_13', 'demand_14', 'demand_15', 'demand_16', 'demand_17', 'demand_18', 'demand_19', 'demand_20', 'demand_21', 'demand_22', 'demand_23', 'demand_24'] not in index"

In [ ]:
state.columns

In [ ]:
state.masked = state.masked.astype(int)

# Masking wind and solar power output

In [ ]:
for i in range(1,25):
    state[f'wind_power_kW_hour_{i}'] *= 1 - state['masked']
    state[f'solar_power_kW_hour_{i}'] *= 1 - state['masked']

In [ ]:
state['installation_type'] = None
state.to_crs("EPSG:32161", inplace=True)

In [ ]:
state.to_parquet('../data/processed/state_1km_grid_cells_masked_power.parquet')

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
def plot_columns(orig_gdf, columns=[]):
    gdf = orig_gdf.to_crs('EPSG:4326')
    if len(columns) == 0:
        columns = gdf.columns

    for column in columns:
        if column in ['geometry', 'centroid']:
            continue

        fig, ax = plt.subplots(1, 1, figsize=(12, 8))

        # Check if the column is binary
        if set(gdf[column].unique()) == {0, 1}:
            # Custom colors for binary data
            light_grey = "#838383"  # Lighter grey
            greenish_blue_green = "#6bc4a9"  # Greenish blue-green
            
            # Create a custom colormap
            cmap = ListedColormap([greenish_blue_green, light_grey])
            #cmap = ListedColormap(['#abd9e9', '#fdae61'])  # Light blue and gentle orange
            gdf.plot(column=column, ax=ax, legend=False, cmap=cmap)
        else:
            # Default plotting for non-binary data
            gdf.plot(column=column, ax=ax, legend=True, legend_kwds={'orientation': 'horizontal', 'label': 'Solar Power Potential (MWh)'})

        plt.title(column)
        # plt.title('Solar Power Potential per Cell at 4pm',size=18)
        # plt.xlabel('Longitude',size=16)
        # plt.ylabel('Latitude',size=16)
        # plt.tight_layout()
        # plt.savefig('solar_power.png', dpi=300)
        plt.show()

In [ ]:
plot_columns(state, ['solar_power_kW_hour_20'])

In [ ]:
plot_columns(state)